In [2]:
import numpy as np
from fetch_data import get_data, remove_cols
from helper import *
from getters_alpaca import *
import torch
import matplotlib.pyplot as plt

In [3]:
coin_name = "LTCUSDT"
one_candle_intervals = "15m"
n_periods: int = 1000 # alpaca api limit

from fetch_data import get_data, remove_cols
# get alpaca data
alpaca_data = get_data(coin_name, one_candle_intervals, n_periods)
alpaca_data = remove_cols(alpaca_data, "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume")

# turn it to np
alpaca_data = np.array(alpaca_data)

In [4]:
# make series with usefull data
data = fill_data(alpaca_data)
#display_data_to_file(data, "working_data")
m, n = data.shape
np.random.shuffle(data)

In [6]:
data_train = data[:800]
y_train = torch.tensor(data_train[:,0], dtype=torch.float32)
x_train = torch.tensor(data_train[:, 1:n], dtype=torch.float32)

data_test = data[800:m]
y_test = torch.tensor(data_test[:,0], dtype=torch.float32)
x_test = torch.tensor(data_test[:, 1:n], dtype=torch.float32)

In [1]:
plt.figure(figsize=(8, 6))
plt.scatter(data[:,8 ]-data[:,5], data[:, 0])
plt.ylabel('Closing Price Change')
plt.show()

NameError: name 'plt' is not defined